# Function Definitions

In [34]:
def convert_df_to_datetime(df, columns_to_convert):
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column])

In [35]:
#turns out to be unnecessary as pd.to_datetime can handle the trailing "UT" in the timestamp string
def clean_timestamp_columns_and_strip_UT(df, columns_to_convert):
    for column in columns_to_convert:
        df[column] = df[column].str.strip("UT")

# Flight Analysis

In [36]:
import pandas as pd
from geopy.distance import distance
from datetime import timedelta
import matplotlib.pyplot as plt
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, LocalTileLayer, Marker, AwesomeIcon, Popup, FullScreenControl
from ipywidgets import HTML, Layout

In [37]:
#pd.read_csv('data/REGA_1.csv', sep=';')

In [38]:
heli = pd.read_csv('data/REGA_12.csv', sep=';').dropna()
heli['Zeit Status DP'] = pd.to_datetime(heli['Zeit Status DP'])
heli['Zeit Status 1'] = pd.to_datetime(heli['Zeit Status 1'])
heli['Zeit Status 2'] = pd.to_datetime(heli['Zeit Status 2'])
heli['Zeit Status 3'] = pd.to_datetime(heli['Zeit Status 3'])
heli['Zeit Status 4'] = pd.to_datetime(heli['Zeit Status 4'])
heli

C:\Users\elsnuc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname UT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Disponummer,Einsatznummer,Einheit,Zeit Status DP,Latitude Disposition,Longitude Disposition,Zeit Status 1,Latitude Status 1,Longitude Status 1,Zeit Status 2,Latitude Status 2,Longitude Status 2,Zeit Status 3,Latitude Status 3,Longitude Status 3,Zeit Status 4,Latitude Status 4,Longitude Status 4
1,D22091729,S22128822,REGA_12,2022-08-27 07:34:46,47.078041,9.06661,2022-08-27 07:35:31,47.078041,9.06661,2022-08-27 07:50:37,47.310875,8.789150,2022-08-27 08:26:07,47.311963,8.785355,2022-08-27 08:34:05,47.377310,8.550802
4,D22091907,S22129070,REGA_12,2022-08-27 16:20:57,47.078041,9.06661,2022-08-27 16:22:17,47.078041,9.06661,2022-08-27 16:37:02,47.198513,8.700391,2022-08-27 16:52:12,47.198513,8.700391,2022-08-27 17:04:45,47.372134,8.557430


In [39]:
einsatz = input("Einsatz: ")
flug = heli[heli['Einsatznummer'] == einsatz]
if flug.empty:
    raise ValueError("Eingegebene Einsatznummer nicht gefunden.")
zeitpunkte = flug[['Zeit Status DP', 'Zeit Status 1', 'Zeit Status 2', 'Zeit Status 3', 'Zeit Status 4']].iloc[0]
lats = flug[['Latitude Disposition', 'Latitude Status 1', 'Latitude Status 2', 'Latitude Status 3','Latitude Status 4']].iloc[0].values
lons = flug[['Longitude Disposition', 'Longitude Status 1', 'Longitude Status 2', 'Longitude Status 3','Longitude Status 4']].iloc[0].values

latlons = list(zip(lats, lons))
zeitpunkte = [zp.strftime("%H:%M:%S") for zp in zeitpunkte]
message0 = HTML()
message1 = HTML()
message2 = HTML()
message3 = HTML()
message4 = HTML()
message0.value = ("<b>Disponiert: </b>") + zeitpunkte[0]
message1.value = ("<b>Ausgerückt: </b><br>") + zeitpunkte[1]
message2.value = ("<b>Am Einsatzort: </b><br>") + zeitpunkte[2]
message3.value = ("<b>Transport Patient: </b><br>") + zeitpunkte[3]
message4.value = ("<b>Am Zielort: </b><br>") + zeitpunkte[4]

line = Polyline(
    locations=latlons,
    color="magenta",
    fill=False
)

popup_0 = Popup(
    location=latlons[0],
    child=message1,
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)

popup_1 = Popup(
    location=latlons[1],
    child=message1,
    close_button=False,
    auto_close=False,
    close_on_escape_key=True
)

popup_2 = Popup(
    location=latlons[2],
    child=message2,
    close_button=False,
    auto_close=False,
    close_on_escape_key=True
)

popup_3 = Popup(
    location=latlons[3],
    child=message3,
    close_button=False,
    auto_close=False,
    close_on_escape_key=True
)

popup_4 = Popup(
    location=latlons[4],
    child=message4,
    close_button=False,
    auto_close=False,
    close_on_escape_key=True
)

start = AwesomeIcon(
    name='home',
    marker_color='blue',
    icon_color='darkblue',
)

ziel = AwesomeIcon(
    name='h-square',
    marker_color='blue',
    icon_color='darkblue',
)

eo = AwesomeIcon(
    name='heartbeat',
    marker_color='blue',
    icon_color='darkblue',
)

route = AwesomeIcon(
    name='road',
    marker_color='blue',
    icon_color='darkblue',
)


marker1 = Marker(icon=start, title = "Ausgerückt", location=latlons[1], draggable=False, popup = message1)
marker2 = Marker(icon=ziel, title = "Am Einsatzort", location=latlons[4], draggable=False, popup = message4)
marker3 = Marker(icon=eo, title = "Transport Patient", location=latlons[2], draggable=False, popup = message2)
marker4 = Marker(icon=route, title = "Am Zielort", location=latlons[3], draggable=False, popup = message3)

# Initialisierung Karte
m = Map(center = latlons[2], zoom =11, layout=Layout(width='100%', height='1050px'), scroll_wheel_zoom=True)
m.add_layer(LocalTileLayer(path='https://tile.osm.ch/switzerland/{z}/{x}/{y}.png'))
m.add_layer(marker1)
m.add_layer(marker2)
m.add_layer(marker3)
m.add_layer(marker4)
m.add_layer(line)
#m.add_layer(popup_0)
#m.add_layer(popup_1)
#m.add_layer(popup_2)
#m.add_layer(popup_3)
#m.add_layer(popup_4)
m.add_control(FullScreenControl())





In [40]:
# Export nach HTML
m.save('analysis.html', title='Fluganalyse')

# Kartenanbieter

OSM Standard Swiss
https://tile.osm.ch/switzerland/{z}/{x}/{y}.png

SRZ Opentopomap Dark
m.add_layer(LocalTileLayer(path='http://elsm1451.els.loc/srz-dark/{z}/{x}/{y}.png'))

SRZ Opentopomap Light
m.add_layer(LocalTileLayer(path='http://elsm1451.els.loc/opentopomap/{z}/{x}/{y}.png'))

OpenTopoMap
opentopomap = basemap_to_tiles(basemaps.OpenTopoMap)
m.add_layer(opentopomap)

Swisstopo Basemap
swisstopo = basemap_to_tiles(basemaps.SwissFederalGeoportal.NationalMapColor)
m.add_layer(swisstopo)

Swisstopo Light Base Map
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

Swisstopo Light Base Map Dark
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm-dark/{z}/{x}/{y}@2x.png?key=62fWxjuKZdoP6vlFjq0a'))

Swisstopo Light Base Map Vivid
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm-vivid/256/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

MapTiler Bright
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/bright/256/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))

Swiss Style SOSM
m.add_layer(LocalTileLayer(path='https://tile.osm.ch/osm-swiss-style/{z}/{x}/{y}.png'))

Streets Style
m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/streets/{z}/{x}/{y}@2x.png?key=62fWxjuKZdoP6vlFjq0a'))



In [41]:
#m = Map(center=(47.25983648568554, 8.71772586165636), zoom=9)
#m.add_layer(LocalTileLayer(path='https://api.maptiler.com/maps/ch-swisstopo-lbm/{z}/{x}/{y}.png?key=62fWxjuKZdoP6vlFjq0a'))
#m
